<a href="https://colab.research.google.com/github/maryamelnahas/CSEN711-MS2-Cryptography-and-Encryption/blob/main/CSEN711_MS2_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import math
import os
from google.colab import files
import hashlib
import secrets
import getpass
import re
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
import hashlib
import binascii

**Loading Dataset**

In [5]:
dataset = pd.read_csv('production.tsv', sep='\t')
print("Columns found:", dataset.columns.tolist())
print(dataset.head())

Columns found: ['password', 'strength']
                                            password  strength
0                                        5Ka392f0c29         1
1  97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjU...         2
2                                         qD2bN8VmEB         1
3                                            rJu42Kx         1
4                                          YloGVMJoL         0


**Playfair Cipher**

In [6]:
SECRET_KEY = "BUSINESSINFORMATICS"

def prepare_text_playfair(text):
    text = str(text).upper().replace("J", "I")
    return re.sub(r'[^A-Z]', '', text)

def generate_key_matrix(key):
    key = prepare_text_playfair(key)
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ" # 'J' is omitted
    matrix = []
    used_chars = set()

    for char in key:
        if char not in used_chars and char in alphabet:
            matrix.append(char)
            used_chars.add(char)

    for char in alphabet:
        if char not in used_chars:
            matrix.append(char)
            used_chars.add(char)

    return [matrix[i:i+5] for i in range(0, 25, 5)]

def find_position(matrix, char):
    for r, row in enumerate(matrix):
        if char in row:
            return r, row.index(char)
    return None

def preprocess_digraphs(text):
    text = prepare_text_playfair(text)
    digraphs = []
    i = 0
    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) else 'X'

        if a == b:
            digraphs.append((a, 'X'))
            i += 1
        else:
            digraphs.append((a, b))
            i += 2
    return digraphs

def encrypt_playfair(plaintext, key):
    matrix = generate_key_matrix(key)
    digraphs = preprocess_digraphs(plaintext)
    ciphertext = []

    for a, b in digraphs:
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2: # Same Row
            ciphertext.append(matrix[r1][(c1 + 1) % 5])
            ciphertext.append(matrix[r2][(c2 + 1) % 5])
        elif c1 == c2: # Same Column
            ciphertext.append(matrix[(r1 + 1) % 5][c1])
            ciphertext.append(matrix[(r2 + 1) % 5][c2])
        else: # Neither
            ciphertext.append(matrix[r1][c2])
            ciphertext.append(matrix[r2][c1])

    return "".join(ciphertext)

# Test

dataset['playfairEncrypted'] = dataset['password'].apply(
    lambda x: encrypt_playfair(x, SECRET_KEY)
)

print(f"Playfair Encryption Results (Key: {SECRET_KEY})\n")
print(dataset[['password', 'playfairEncrypted']].head(10).to_markdown(index=False))

Playfair Encryption Results (Key: BUSINESSINFORMATICS)

| password                                                                                           | playfairEncrypted                                                    |
|:---------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------|
| 5Ka392f0c29                                                                                        | HTOT                                                                 |
| 97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7 | GRNCNHHQKVRWYSYZWRXEKVEVMEIBORIENSNYSYSYQGEYOPYERCDRPGOEUNNUQFPULSZQ |
| qD2bN8VmEB                                                                                         | PGUBZEAE                                                             |
| rJu42Kx                                                                 

**Vigenère Cipher**

In [7]:
def encrypt_vigenere(plaintext, key):
    plaintext = re.sub(r'[^A-Z]', '', str(plaintext).upper())
    key = re.sub(r'[^A-Z]', '', str(key).upper())
    if not plaintext:
        return ""
    ciphertext = []
    key_length = len(key)
    key_as_int = [ord(i) - 65 for i in key]
    plaintext_int = [ord(i) - 65 for i in plaintext]
    for i, val in enumerate(plaintext_int):
        key_val = key_as_int[i % key_length]
        enc_val = (val + key_val) % 26
        ciphertext.append(chr(enc_val + 65))
    return "".join(ciphertext)

# Test

SECRET_KEY = "BUSINESSINFORMATICS"
dataset['vigenereEncrypted'] = dataset['password'].apply(
    lambda x: encrypt_vigenere(x, SECRET_KEY)
)

print(f"Vigenère Encryption Results (Key: {SECRET_KEY})\n")
print(dataset[['password', 'vigenereEncrypted']].head(10).to_markdown(index=False))

Vigenère Encryption Results (Key: BUSINESSINFORMATICS)

| password                                                                                           | vigenereEncrypted                                                   |
|:---------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------|
| 5Ka392f0c29                                                                                        | LUXK                                                                |
| 97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7 | EGKOOUIHPJKMOVXRGHNPBOJUVEKVSTPIVUBHLBYAEZIVDNZBIFACDYUDAKVCUXNUPYT |
| qD2bN8VmEB                                                                                         | RXTVIQWT                                                            |
| rJu42Kx                                                                      

**AES-256**

In [8]:
def get_aes_key(passphrase):
    return hashlib.sha256(passphrase.encode()).digest()

def encrypt_aes(plaintext, key_bytes):
    iv = get_random_bytes(16)
    cipher = AES.new(key_bytes, AES.MODE_CBC, iv)
    padded_data = pad(str(plaintext).encode(), AES.block_size)
    ciphertext = cipher.encrypt(padded_data)
    return binascii.hexlify(iv + ciphertext).decode('utf-8')

# Test

passphrase = "BUSINESSINFORMATICS"
aes_key = get_aes_key(passphrase)
dataset['AESEncrypted'] = dataset['password'].apply(
    lambda x: encrypt_aes(x, aes_key)
)

print(f"AES-256 Encryption Results (Passphrase: {passphrase})\n")
print(dataset[['password', 'AESEncrypted']].head(10).to_markdown(index=False))

AES-256 Encryption Results (Passphrase: BUSINESSINFORMATICS)

| password                                                                                           | AES_Encrypted                                                                                                                                                                                                                                                    |
|:---------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 5Ka392f0c29                                                                                        | 36f15d2c2b5c8a56882ef71b4907af1abfcd85c08cb2786abe6fc738e356de66                                               